In [22]:
import numpy as np
from arrays import age_array

# this function produces a two-dimensional numpy array of shape 2 x n, where n represents
# the total number of periods in the projection. the first row represents the age array beginning 
# at the starting age and ending after n years. the second row represents the allocation array that 
# defines the pct of the investment fund to be invested in stocks at the corresponding age.
def allocation_array(
    start_pct = .90,        # pct invested in equities at the start of the projection
    five_yr_pct = .55,      # pct invested in equities 5 years out from retirement
    end_pct = .40,          # pct invested in equities at retirement
    age = 30,               # age at the start of the projection
    ret_age = 65,           # retirement age and the age at which the end allocation pct is reached
    periods_per_year = 2,   # the number of periods in a year
    years = 40              # length of the projection in years
):
    
    # income_period represents the number of income earning years
    income_period = ret_age - age

    # segment_one_length represents the number of periods at the initial constant allocation
    segment_one_length = 0 if income_period <= 30 else (income_period - 30) * periods_per_year

    # segment_four_length represents the number of periods at the ending constant allocation
    segment_four_length = 0 if years <= income_period else (years - income_period) * periods_per_year + 1

    # calculate segment_two_length, start pct and end pct
    # segment two represents the period from 30 to 5 years from retirement
    segment_two_slope = (five_yr_pct - start_pct) / (25 * periods_per_year)
    segment_two_end_pct = five_yr_pct - segment_two_slope
    if income_period <= 5:
        segment_two_length = 0
        segment_two_start_pct = 0
    elif income_period >= 30:
        segment_two_length = 25 * periods_per_year
        segment_two_start_pct = start_pct 
    else:
        segment_two_length = (income_period - 5) * periods_per_year
        segment_two_start_pct = five_yr_pct + ((income_period - 5) / 25 * (start_pct - five_yr_pct))

    # calculate segment_three_length, start pct and end pct
    # segment three represents the period from 5 to 0 years from retirement
    segment_three_slope = (end_pct - five_yr_pct) / (5 * periods_per_year)
    segment_three_end_pct = end_pct - segment_three_slope
    if income_period >= 5:
        segment_three_length = 5 * periods_per_year
        segment_three_start_pct = five_yr_pct       
    else:
        segment_three_length = income_period * periods_per_year
        segment_three_start_pct = end_pct + (income_period / 5 * (five_yr_pct - end_pct))

    # setup allocation arrays
    segment_one_array = np.full(segment_one_length, start_pct)
    segment_two_array = np.linspace(segment_two_start_pct, segment_two_end_pct, segment_two_length)
    segment_three_array = np.linspace(segment_three_start_pct, segment_three_end_pct, segment_three_length)
    segment_four_array = np.full(segment_four_length, end_pct)

    # concatenate the arrays into one single array
    equity_allocation_array = np.concatenate((segment_one_array,segment_two_array,segment_three_array,segment_four_array))
    bond_allocation_array = 1 - equity_allocation_array

    # create an age array to act as a helper array when graphing
    age_arr = age_array(age = age, periods_per_year = periods_per_year, years = years)

    # add the age, equity and bond allocation arrays into a single multidimensional array
    result_array = np.vstack((age_arr, equity_allocation_array, bond_allocation_array))

    return result_array

result = allocation_array()
print(result[1])

[0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.893
 0.886 0.879 0.872 0.865 0.858 0.851 0.844 0.837 0.83  0.823 0.816 0.809
 0.802 0.795 0.788 0.781 0.774 0.767 0.76  0.753 0.746 0.739 0.732 0.725
 0.718 0.711 0.704 0.697 0.69  0.683 0.676 0.669 0.662 0.655 0.648 0.641
 0.634 0.627 0.62  0.613 0.606 0.599 0.592 0.585 0.578 0.571 0.564 0.557
 0.55  0.535 0.52  0.505 0.49  0.475 0.46  0.445 0.43  0.415 0.4   0.4
 0.4   0.4   0.4   0.4   0.4   0.4   0.4   0.4   0.4  ]


In [ ]:
    # result_df = pd.DataFrame(result)

    # allocation_array_df = pd.DataFrame(allocation_array)
    # segment_two_array_df = pd.DataFrame(segment_two_array)
    # age_array_df = pd.DataFrame(age_array)

    # with pd.ExcelWriter('allocation_array.xlsx') as writer:
    #     allocation_array_df.to_excel(writer, sheet_name='allocation_array')
    #     segment_two_array_df.to_excel(writer, sheet_name='segment_two_array')
    #     age_array_df.to_excel(writer, sheet_name='age_array')

    # print(age_array.shape)


# df = allocation_array()
# print(df)
# sns.lineplot(x = df.iloc[0].values, y = df.iloc[1].values)
# plt.show()